<center><h1>Geng_Haoxiang_HW3</h1></center>
<br>
<br>

Name: Haoxiang Geng
<br>
Github Username: Haoxiang310
<br>
USC ID: 8045015278

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [10]:
import os
import pandas as pd
import csv
import numpy as np

Get the AReM Data Set

In [11]:
folder_paths = {
    'bending1': '../data/AReM/bending1',
    'bending2': '../data/AReM/bending2',
    'cycling': '../data/AReM/cycling',
    'lying': '../data/AReM/lying',
    'sitting': '../data/AReM/sitting',
    'standing': '../data/AReM/standing',
    'walking': '../data/AReM/walking'
}

sniffer = csv.Sniffer()

def get_datasets_in_folder(folder_path):
    datasets = os.listdir(folder_path)
    datasets.sort(key=lambda x: int(x.split('dataset')[1].split('.')[0]))
    return datasets
        
columns = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"]

#fix the error dataset(cycling/dataset9.csv and cycling/dataset14.csv)
def fix_error_dataset(dataset_path):
    if "cycling/dataset9.csv" in dataset_path or "cycling/dataset14.csv" in dataset_path:
        with open(dataset_path, 'r') as f:
            lines = f.readlines()
            lines[-1] = lines[-1][:-1]
        
        with open(dataset_path, 'w') as f:
            f.writelines(lines)

def read_dataset_file(dataset_path):
    fix_error_dataset(dataset_path)
    ##fix the error dataset(bending2/dataset4.csv)
    if 'bending2/dataset4' in dataset_path:
        delimiter = '\s+'
    else:
        delimiter = ','
    data = pd.read_csv(dataset_path, sep=delimiter, skiprows=5, header=None, names=columns)
    data['label'] = os.path.basename(os.path.dirname(dataset_path))
    return data


### (b) Test and Train Data

In [12]:
test_data = []
train_data = []
test_data_paths = []
train_data_paths = []

for folder, path in folder_paths.items():
    datasets = get_datasets_in_folder(path)
    if folder in ['bending1', 'bending2']:
        test_datasets = datasets[0:2]
    else:
        test_datasets = datasets[0:3]

    for dataset in datasets:
        dataset_path = os.path.join(path, dataset)
        data = read_dataset_file(dataset_path)
        
        if data is not None:
            dataset_name = os.path.join(folder, dataset)
            if dataset in test_datasets:
#                 print(f"Test data for {dataset_name}:","\n", data)
                test_data.append(data)
                test_data_paths.append(dataset_name)  
            else:
#                 print(f"Train data for {dataset_name}:","\n", data)
                train_data.append(data)
                train_data_paths.append(dataset_name)

print("Datasets in train set:",train_data_paths,"\n")
print("Datasets in test set:",test_data_paths)

Datasets in train set: ['bending1/dataset3.csv', 'bending1/dataset4.csv', 'bending1/dataset5.csv', 'bending1/dataset6.csv', 'bending1/dataset7.csv', 'bending2/dataset3.csv', 'bending2/dataset4.csv', 'bending2/dataset5.csv', 'bending2/dataset6.csv', 'cycling/dataset4.csv', 'cycling/dataset5.csv', 'cycling/dataset6.csv', 'cycling/dataset7.csv', 'cycling/dataset8.csv', 'cycling/dataset9.csv', 'cycling/dataset10.csv', 'cycling/dataset11.csv', 'cycling/dataset12.csv', 'cycling/dataset13.csv', 'cycling/dataset14.csv', 'cycling/dataset15.csv', 'lying/dataset4.csv', 'lying/dataset5.csv', 'lying/dataset6.csv', 'lying/dataset7.csv', 'lying/dataset8.csv', 'lying/dataset9.csv', 'lying/dataset10.csv', 'lying/dataset11.csv', 'lying/dataset12.csv', 'lying/dataset13.csv', 'lying/dataset14.csv', 'lying/dataset15.csv', 'sitting/dataset4.csv', 'sitting/dataset5.csv', 'sitting/dataset6.csv', 'sitting/dataset7.csv', 'sitting/dataset8.csv', 'sitting/dataset9.csv', 'sitting/dataset10.csv', 'sitting/dataset11

### (c) Feature Extraction

#### i. Research

Mean: The average value of the time series data.   
Standard Deviation: The variability of the data.   
Minimum: The smallest value in the data.   
Maximum: The largest value in the data.   
Median: The middle value of the data.   
Quartiles: Represent the 25th, 50th, and 75th percentiles.     
Zero Crossing Rate: The number of times the time series goes from positive to negative or vice versa.   
Peak Values: The local maximum values in the time series.

#### ii. Extraction

In [13]:
def extract_features(data):
    extracted_features = []
    
    for instance in data:
        extracted_feature_values = []
        for column in columns[1:]:
            extracted_feature_values.extend([
                instance[column].min(),
                instance[column].max(),
                instance[column].mean(),
                instance[column].median(),
                instance[column].std(),
                instance[column].quantile(0.25),
                instance[column].quantile(0.75)
            ])
        extracted_features.append(extracted_feature_values)
        
    return extracted_features

stats_features = ['min', 'max', 'mean', 'median', 'std', '1st_quartile', '3rd_quartile']
new_train_data = extract_features(train_data)
new_test_data = extract_features(test_data)
new_columns = [f"{stats_feature}{i}" for i in range(1, 7) for stats_feature in stats_features]
train_set_df = pd.DataFrame(new_train_data, columns=new_columns)
test_set_df = pd.DataFrame(new_test_data, columns=new_columns)
combined_df = pd.concat([train_set_df, test_set_df], axis=0).reset_index(drop=True)
print("Extraced features are:\n", combined_df)

Extraced features are:
      min1   max1      mean1  median1      std1  1st_quartile1  3rd_quartile1  \
0   35.00  47.40  43.954500   44.330  1.558835          43.00        45.0000   
1   33.00  47.75  42.179812   43.500  3.670666          39.15        45.0000   
2   33.00  45.75  41.678063   41.750  2.243490          41.33        42.7500   
3   37.00  48.00  43.454958   43.250  1.386098          42.50        45.0000   
4   36.25  48.00  43.969125   44.500  1.618364          43.31        44.6700   
..    ...    ...        ...      ...       ...            ...            ...   
83  35.50  46.25  43.174938   43.670  1.989052          42.50        44.5000   
84  32.75  47.00  42.760562   44.500  3.398919          41.33        45.3725   
85  19.33  43.50  34.227771   35.500  4.889576          30.50        37.7500   
86  12.50  45.00  33.509729   34.125  4.850923          30.50        36.7500   
87  15.00  46.75  34.660583   35.000  5.315110          31.00        38.2500   

    min2   max2

#### iii. Standard Deviation

In [14]:
combined_df.describe().loc['std']

min1             9.569975
max1             4.394362
mean1            5.335700
median1          5.440054
std1             1.772185
1st_quartile1    6.153874
3rd_quartile1    5.138925
min2             0.000000
max2             5.062729
mean2            1.574198
median2          1.412293
std2             0.884137
1st_quartile2    0.946386
3rd_quartile2    2.125399
min3             2.956462
max3             4.875137
mean3            4.008228
median3          4.036396
std3             0.946670
1st_quartile3    4.220658
3rd_quartile3    4.171628
min4             0.000000
max4             2.183625
mean4            1.166178
median4          1.145985
std4             0.458283
1st_quartile4    0.843405
3rd_quartile4    1.552504
min5             6.124001
max5             5.741238
mean5            5.675543
median5          5.813782
std5             1.024918
1st_quartile5    6.096465
3rd_quartile5    5.531720
min6             0.045838
max6             2.518921
mean6            1.154889
median6     

In [15]:
def bootstrap_confidence_interval(original_data):

    sample_std_list = []

    #number of samples is 10000
    for _ in range(10000):
        sample = np.random.choice(original_data, size=len(original_data), replace=True)
        sample_std = sample.std()
        sample_std_list.append(sample_std)

    np_std_samples = np.array(sample_std_list)
    lower_bound = np.percentile(np_std_samples, 5)
    upper_bound = np.percentile(np_std_samples, 95)
    
    return lower_bound, upper_bound

confidence_intervals = {} #(feature: CI)
for col in combined_df.columns:
    lower_bound, upper_bound = bootstrap_confidence_interval(combined_df[col])
    confidence_intervals[col] = [lower_bound, upper_bound]

for feature, interval in confidence_intervals.items():
    print(f"90% confidence interval for feature {feature} is: {interval}")

90% confidence interval for feature min1 is: [8.218431315273135, 10.688672423845563]
90% confidence interval for feature max1 is: [3.3120507121583405, 5.250534926085106]
90% confidence interval for feature mean1 is: [4.680619926011611, 5.851535137285669]
90% confidence interval for feature median1 is: [4.773741363850848, 5.965407461514533]
90% confidence interval for feature std1 is: [1.5593194565169064, 1.9413093339222156]
90% confidence interval for feature 1st_quartile1 is: [5.534957064316321, 6.621268847008651]
90% confidence interval for feature 3rd_quartile1 is: [4.306713918335115, 5.804118947723934]
90% confidence interval for feature min2 is: [0.0, 0.0]
90% confidence interval for feature max2 is: [4.603276917293157, 5.378413548962029]
90% confidence interval for feature mean2 is: [1.389763567344486, 1.6993974363647015]
90% confidence interval for feature median2 is: [1.2319185180334289, 1.5339535906077308]
90% confidence interval for feature std2 is: [0.800710727475747, 0.9375

#### iv. Select Features

In my opinion, the 3 most important time-domain features are **mean**, **min** and **max**. Choosing mean is because we often use mean to represent the typical case of a feature, and the mean is often used to represent the regular case of a random variable. Choosing min and max is because it can give us the range of the data, showing the lower and upper bound of the data(and sometimes, it can represent the extreme case).

## 2. ISLR 3.7.4

### (a) Linear Train

As for traning RSS, the cubic regression model will be lower, since it contains non-linear variables and has more predictors, so that is more flexible than a linear regression and can fit more complex shapes in the data.

### (b) Linear Test

The linear model will perform better than cubic regression model. For training data, the cubic regression will have less bias. However, if the true model is actually linear, which means the cubic regression which have more non-linear predictors will potentially overfit the model from training data, and result in higher test error.

### (c) Not Linear Train

The cubic regression model will preform better than linear model in the aspect of training RSS, since it has non-linear variables and more predictors, which leads to less bias in training process.

### (d) Not Linear Testing

The evidence is not sufficient to judge which model will perform better in the aspect of test RSS. If the non-linearity is mild and quite different from cubic regression, the linear model will perform better; however if the unknown pattern of data is quite similar to cubic relation, then cubic regression will perform better. 

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 